In [13]:
import itertools
import json

In [14]:
# Let's see what fields are present in the card data, so we can accurately model the data and load it into a database.
with open('data/cards.json') as f:
    data = json.load(f)

# Keys are card sets, which also appear in individual card data in the `cardSet` field.
data.keys()

dict_keys(['Basic', 'The League of Explorers', 'Hero Skins', 'Mean Streets of Gadgetzan', 'Debug', 'Naxxramas', 'System', 'Missions', 'The Grand Tournament', 'Tavern Brawl', 'Whispers of the Old Gods', 'Goblins vs Gnomes', 'Credits', 'Karazhan', 'Blackrock Mountain', 'Reward', 'Promo', 'Classic'])

In [19]:
# This yields an iterator that returns one card at a time.
cards = itertools.chain.from_iterable(data.values())
cards = list(cards)

# You won't see most of these cards in a deck.
# They include heroes, characters from the campaign, enchantments, and more.
len(cards)

2818